In [ ]:
pip install psycopg2-binary

In [ ]:
pip install sqlalchemy

In [ ]:
pip install boto3

In [ ]:
pip install selenium

In [1]:
from selenium import webdriver 
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import time
import uuid
import os
import json
import unittest
import pandas as pd
import web_scraper
from pathlib import Path

In [ ]:
def cookies() -> webdriver.Chrome:
    driver = webdriver.Chrome()
    URL = 'https://www.asos.com/men/new-in/new-in-clothing/cat/?cid=6993&nlid=mw|clothing|shop+by+product|new+in'
    driver.get(URL)
    time.sleep(3)
    try:
        accept_cookies_button = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
        accept_cookies_button.click()
        time.sleep(1)
    except AttributeError: # If you have the latest version of selenium, the code above won't run because the "switch_to_frame" is deprecated
        driver.switch_to.frame('onetrust-consent-sdk') # This is the id of the frame
        accept_cookies_button = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
        accept_cookies_button.click()
        time.sleep(1)

    except:
        pass

    return driver 

In [ ]:
cookies()

In [ ]:
def get_links(driver: webdriver.Chrome) ->list:
    driver = cookies()
    clothes_container = driver.find_element_by_xpath('//*[@class="RIQzlgo"]')
    clothes_list = clothes_container.find_elements_by_xpath('./article')
    link_list = []

    for clothes in clothes_list:
        a_tag = clothes.find_element_by_tag_name('a')
        link = a_tag.get_attribute('href')
        link_list.append(link)
        
    print(f'There are {len(link_list)} items of clothing in this page')
    return link_list

In [ ]:
get_links(cookies)

In [ ]:
def create_id():
    id_list = []
    for id in range (72):
        id = str(uuid.uuid4())
        id_list.append(id)
    return id_list

In [ ]:
def get_data():
    clothes = []
    description_list = []
    price_list = []
    colour_list = []
    driver = cookies()
    clothes.extend(get_links(driver))

    for links in clothes:
        driver.get(links)
        description = driver.find_element_by_tag_name('h1').text
        description_list.append(description)
        price = driver.find_element_by_xpath('//span[@data-id="current-price"]').text
        price_list.append(price)
        colour = driver.find_element_by_xpath('//span[@class="product-colour"]').text
        colour_list.append(colour)
    
    return price_list, colour_list, description_list

In [ ]:
def create_dict(id_list, description, price, colour):
    id_dict = {}

    for i in id_list:
        id_dict[i] = [description, price, colour]

    return id_dict

In [ ]:
create_dict(id_list, description, price, colour)

In [ ]:
clothes = []
driver = cookies()
clothes.extend(get_links(driver))

id_dict = {}


for links in clothes:
    driver.get(links)
    for i in id_list :
        description = driver.find_element_by_tag_name('h1').text
        price = driver.find_element_by_xpath('//span[@data-id="current-price"]').text
        colour = driver.find_element_by_xpath('//span[@class="product-colour"]').text
        id_dict[i] = [description, price, colour] 

In [ ]:
def __get_data(self):
    clothes = []
    clothes.extend(self.get_links())

    clothes_list = []

    for links in clothes:
        self.driver.get(links)
        description = self.driver.find_element_by_tag_name('h1').text
        price = self.driver.find_element_by_xpath('//span[@data-id="current-price"]').text
        colour = self.driver.find_element_by_xpath('//span[@class="product-colour"]').text
        id = self.create_id()
        product_dict = self.__create_dict(id, description, price, colour)
        clothes_list.append(product_dict)
    
    return clothes_list

In [ ]:
def create_file():
    directory = 'raw_data'
    parent_dir = '/Users/wadirmalik/Desktop/Data_Collection_Pipeline'
    directory2 = 'Asos'

    path = os.path.join(parent_dir, directory, directory2)

    os.mkdir(path)



In [ ]:
def save_to_file():    
    directory2 = open(path + "/id_dict.json", "w")
    directory2 = json.dump(id_dict, directory2)

In [ ]:
class web_scraperTestCase(unittest.TestCase):

    def test_get_links(self):
        self.assertEqual(len(clothes), 72)

    def test_create_id(self):
        self.assertEqual(len(id_list), 72)

    def test_get_data(self):
        self.assertEqual(len(price_list), len(colour_list), len(description_list))

    def test_create_dict(self):
        self.assertIsInstance(id_dict, dict)

unittest.main(argv=[''], verbosity=0, exit=False)

In [ ]:
import boto3 
s3_client = boto3.client('s3')

In [ ]:
response = s3_client.upload_file('/Users/wadirmalik/Desktop/Data_Collection_Pipeline/raw_data/Asos/id_dict.json', 'scraperwadir', 'id_dict')

In [ ]:
s3 = boto3.resource('s3')

my_bucket = s3.Bucket('scraperwadir')

for file in my_bucket.objects.all():
    print(file.key)

In [ ]:
import requests
# Change this with your URL
url = 'https://scraperwadir.s3.us-east-2.amazonaws.com/id_dict'

response = requests.get(url)
with open('id_dict', 'wb') as f:
    f.write(response.content)

In [1]:
from importlib.resources import path
import web_scraper

asos_scraper = web_scraper.Scraper()
path, clothes_list = asos_scraper.run_scraper()
#asos_scraper.save_to_file(path, clothes_list)

There are 72 items of clothing in this page
There are 72 items of clothing in this page


In [4]:
from sqlalchemy import create_engine

RDS_PASSWORD = 'afgangalo12'
RDS_HOST = 'asos-scraper.c4xuwbscrnlq.us-east-2.rds.amazonaws.com'
RDS_PORT = '5432' 

engine = create_engine(f'postgresql+psycopg2://postgres:{RDS_PASSWORD}@{RDS_HOST}:{RDS_PORT}/postgres')

In [5]:
from sqlalchemy import inspect

inspect(engine)

In [6]:
df = pd.DataFrame(clothes_list)
df.to_sql('clothes_list', engine, if_exists='append')


NameError: name 'pd' is not defined